In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [4]:
fashion = keras.datasets.fashion_mnist

In [5]:
(train_img,train_label) ,(test_img,test_label) = fashion.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_img = train_img/255.0
test_img = test_img/255.0

In [7]:
train_img.shape

(60000, 28, 28)

In [8]:
test_img.shape

(10000, 28, 28)

In [9]:
############ 28 X 28 is my imput image

In [10]:
len(train_img)

60000

In [11]:
train_img = train_img.reshape(len(train_img),28,28,1)
test_img = test_img.reshape(len(test_img),28,28,1)

In [12]:
train_img.shape

(60000, 28, 28, 1)

In [13]:
train_label.shape

(60000,)

In [14]:
train_label[0:15]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7], dtype=uint8)

In [15]:
test_label[0:15]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4], dtype=uint8)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [17]:
!pip install keras-tuner

In [18]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [19]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [20]:
tuner_search.search(train_img,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8765000104904175

Best val_accuracy So Far: 0.9083333611488342
Total elapsed time: 00h 04m 10s
INFO:tensorflow:Oracle triggered exit


In [21]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                2359360   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 2,406,954
Trainable params: 2,406,954
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_img, train_label, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1314 - accuracy: 0.9518 - val_loss: 0.2664 - val_accuracy: 0.9110
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0934 - accuracy: 0.9652 - val_loss: 0.3163 - val_accuracy: 0.9122
Epoch 6/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0638 - accuracy: 0.9773 - val_loss: 0.3488 - val_accuracy: 0.9037
Epoch 7/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0470 - accuracy: 0.9828 - val_loss: 0.4397 - val_accuracy: 0.9063
Epoch 8/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0362 - accuracy: 0.9871 - val_loss: 0.4513 - val_accuracy: 0.9092
Epoch 9/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0287 - accuracy: 0.9896 - val_loss: 0.5175 - val_accuracy: 0.9060
Epoch 10/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0214 - accuracy: 0.9923 - val_loss: 0.5867 - val_accur

In [27]:
pred = model.predict_classes(test_img)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
pred[0:15]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 5, 3, 4])

In [31]:
test_label[0:15]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4], dtype=uint8)